# 1. From products.db:

In [48]:
import sqlite3 as lite
import pandas as pd

In [49]:
db = lite.connect('../resource/products.db')

In [50]:
pd.options.display.max_colwidth = 200

In [51]:
# Inspect tables, schema
query = """
SELECT name, sql 
FROM sqlite_master
;
"""

pd.read_sql(query, db)

,name,sql
0,Categories,"CREATE TABLE Categories(\n ""CategoryID"" TEXT,\n ""CategoryName"" TEXT,\n ""Description"" TEXT\n)"
1,Customers,"CREATE TABLE Customers(\n ""CustomerID"" TEXT,\n ""CustomerName"" TEXT,\n ""ContactName"" TEXT,\n ""Address"" TEXT,\n ""City"" TEXT,\n ""PostalCode"" TEXT,\n ""Country"" TEXT\n)"
2,Employees,"CREATE TABLE Employees(\n ""EmployeeID"" TEXT,\n ""LastName"" TEXT,\n ""FirstName"" TEXT,\n ""BirthDate"" TEXT,\n ""Photo"" TEXT,\n ""Notes"" TEXT\n)"
3,Shippers,"CREATE TABLE Shippers(\n ""ShipperID"" TEXT,\n ""ShipperName"" TEXT,\n ""Phone"" TEXT\n)"
4,Suppliers,"CREATE TABLE Suppliers(\n ""SupplierID"" TEXT,\n ""SupplierName"" TEXT,\n ""ContactName"" TEXT,\n ""Address"" TEXT,\n ""City"" TEXT,\n ""PostalCode"" TEXT,\n ""Country"" TEXT,\n ""Phone"" TEXT\n)"
5,Orders,"CREATE TABLE Orders(\nOrderID integer,\nCustomerID integer,\nEmployeeID integer,\nOrderDate date,\nShipperID integer)"
6,OrderDetails,"CREATE TABLE OrderDetails(\nOrderDetailID integer,\nOrderID integer,\nProductID integer,\nQuantity integer)"
7,Products,"CREATE TABLE Products(\nProductID integer,\nProductName text,\nSupplierID integer,\nCategoryID integer,\nUnit text,\nPrice integer\n)"


## 1) Get the distribution of birth dates of employees

In [52]:
# inspect Employees table
query = """
SELECT *
FROM Employees
;
"""

pd.read_sql(query, db)

,EmployeeID,LastName,FirstName,BirthDate,Photo,Notes
0,1,Davolio,Nancy,1968-12-08,EmpID1.pic,Education includes a BA in psychology from Colorado State University. She also completed (The Art of the Cold Call). Nancy is a member of 'Toastmasters International'.
1,2,Fuller,Andrew,1952-02-19,EmpID2.pic,Andrew received his BTS commercial and a Ph.D. in international marketing from the University of Dallas. He is fluent in French and Italian and reads German. He joined the company as a sales repre...
2,3,Leverling,Janet,1963-08-30,EmpID3.pic,Janet has a BS degree in chemistry from Boston College). She has also completed a certificate program in food retailing management. Janet was hired as a sales associate and was promoted to sales r...
3,4,Peacock,Margaret,1958-09-19,EmpID4.pic,Margaret holds a BA in English literature from Concordia College and an MA from the American Institute of Culinary Arts. She was temporarily assigned to the London office before returning to her p...
4,5,Buchanan,Steven,1955-03-04,EmpID5.pic,Steven Buchanan graduated from St. Andrews University Scotland with a BSC degree. Upon joining the company as a sales representative he spent 6 months in an orientation program at the Seattle o...
5,6,Suyama,Michael,1963-07-02,EmpID6.pic,Michael is a graduate of Sussex University (MA economics) and the University of California at Los Angeles (MBA marketing). He has also taken the courses 'Multi-Cultural Selling' and 'Time Manage...
6,7,King,Robert,1960-05-29,EmpID7.pic,Robert King served in the Peace Corps and traveled extensively before completing his degree in English at the University of Michigan and then joining the company. After completing a course entitle...
7,8,Callahan,Laura,1958-01-09,EmpID8.pic,Laura received a BA in psychology from the University of Washington. She has also completed a course in business French. She reads and writes French.
8,9,Dodsworth,Anne,1969-07-02,EmpID9.pic,Anne has a BA degree in English from St. Lawrence College. She is fluent in French and German.
9,10,West,Adam,1928-09-19,EmpID10.pic,An old chum.


In [53]:
#answer
query = """
SELECT substr(E.BirthDate, 6, 2) "Month", count(*) "Count"
FROM Employees E
    GROUP BY
        "Month"
;
"""

pd.read_sql(query, db)

,Month,Count
0,01,1
1,02,1
2,03,1
3,05,1
4,07,2
5,08,1
6,09,2
7,12,1


In [54]:
#위에 비어있는 달은 0으로 해서 추가할수없을까?

## 2) Get the sum of sales revenue per OrderID

In [55]:
# inspect OrderDetails table
query = """
SELECT *
FROM OrderDetails
;
"""

pd.read_sql(query, db).head()

,OrderDetailID,OrderID,ProductID,Quantity
0,1,10248,11,12
1,2,10248,42,10
2,3,10248,72,5
3,4,10249,14,9
4,5,10249,51,40


In [56]:
# inspect Products table
query = """
SELECT *
FROM Products
;
"""

pd.read_sql(query, db).head()

,ProductID,ProductName,SupplierID,CategoryID,Unit,Price
0,1,Chais,1,1,10 boxes x 20 bags,18.00
1,2,Chang,1,1,24 - 12 oz bottles,19.00
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35


In [57]:
# JOIN OrderDetails and Products on ProductID
query = """
SELECT *
FROM OrderDetails D
    JOIN
        Products P
    ON 
        D.ProductID = P.ProductID
;
"""

pd.read_sql(query, db).head()

,OrderDetailID,OrderID,ProductID,Quantity,ProductID,ProductName,SupplierID,CategoryID,Unit,Price
0,1,10248,11,12,11,Queso Cabrales,5,4,1 kg pkg.,21.00
1,2,10248,42,10,42,Singaporean Hokkien Fried Mee,20,5,32 - 1 kg pkgs.,14.00
2,3,10248,72,5,72,Mozzarella di Giovanni,14,4,24 - 200 g pkgs.,34.80
3,4,10249,14,9,14,Tofu,6,7,40 - 100 g pkgs.,23.25
4,5,10249,51,40,51,Manjimup Dried Apples,24,7,50 - 300 g pkgs.,53.00


In [59]:
#answer
query = """
SELECT D.OrderID, sum(D.Quantity * P.Price) "Revenue"
FROM OrderDetails D
    JOIN
        Products P
    ON 
        D.ProductID = P.ProductID
    GROUP BY
        OrderID
;
"""

pd.read_sql(query, db).head()

,OrderID,Revenue
0,10248,566.00
1,10249,2329.25
2,10250,2267.25
3,10251,839.50
4,10252,4662.50


# 2. On Zigbang

In [19]:
import requests
from pymongo import MongoClient

In [21]:
mongo_uri = "mongodb://<mLabusername>:<mLabpassword>@ds145299.mlab.com:45299/mydbinstance"

In [22]:
client = MongoClient(mongo_uri) 

In [23]:
client.mydbinstance.collection_names()

c:\python37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['bigbang', 'system.indexes', 'users', 'nvkwlist']

In [24]:
db = client.mydbinstance

In [25]:
bigbang_collection = db.bigbang

## 1 ) Get the list of data WHERE total floor >= 4 and floor >= 3

In [26]:
# inspect the data
bigbang_collection.find_one({})

{'_id': ObjectId('5c399b2ea6bb5b6cc4db1d39'),
 'title': '세종시 조치원읍 침산리',
 'header': False,
 'header_height': 0,
 'item': {'id': 14187437,
  'images': [{'index': 0,
    'count': 1,
    'url': 'https://ic.zigbang.com/ic/items/14187437/1.jpg'},
   {'index': 1,
    'count': 2,
    'url': 'https://ic.zigbang.com/ic/items/14187437/2.jpg'},
   {'index': 2,
    'count': 3,
    'url': 'https://ic.zigbang.com/ic/items/14187437/3.jpg'},
   {'index': 3,
    'count': 4,
    'url': 'https://ic.zigbang.com/ic/items/14187437/4.jpg'},
   {'index': 4,
    'count': 5,
    'url': 'https://ic.zigbang.com/ic/items/14187437/5.jpg'},
   {'index': 5,
    'count': 6,
    'url': 'https://ic.zigbang.com/ic/items/14187437/6.jpg'},
   {'index': 6,
    'count': 7,
    'url': 'https://ic.zigbang.com/ic/items/14187437/7.jpg'}],
  'is_realestate': True,
  'is_direct': False,
  'is_room': False,
  'is_type_room': False,
  'rent': 23,
  'deposit': 100,
  'is_deposit_only': False,
  'floor': '3층',
  '_floor': '3층',
  'floo

In [29]:
query = {}

bigbang_collection.find_one(query)["item"]["floor_all"][0]

'4'

In [30]:
query = {}

bigbang_collection.find_one(query)["item"]["floor"][0]

'3'

In [42]:
# answer : select data where floor_all >= 4 and floor >= 3

query = {}

[bang['item'] for bang in list(bigbang_collection.find(query)) if int(bang['item']['floor_all'][0]) >= 4 and int(bang['item']['floor'][0].replace("반", "0")) >= 3]

[{'id': 14187437,
  'images': [{'index': 0,
    'count': 1,
    'url': 'https://ic.zigbang.com/ic/items/14187437/1.jpg'},
   {'index': 1,
    'count': 2,
    'url': 'https://ic.zigbang.com/ic/items/14187437/2.jpg'},
   {'index': 2,
    'count': 3,
    'url': 'https://ic.zigbang.com/ic/items/14187437/3.jpg'},
   {'index': 3,
    'count': 4,
    'url': 'https://ic.zigbang.com/ic/items/14187437/4.jpg'},
   {'index': 4,
    'count': 5,
    'url': 'https://ic.zigbang.com/ic/items/14187437/5.jpg'},
   {'index': 5,
    'count': 6,
    'url': 'https://ic.zigbang.com/ic/items/14187437/6.jpg'},
   {'index': 6,
    'count': 7,
    'url': 'https://ic.zigbang.com/ic/items/14187437/7.jpg'}],
  'is_realestate': True,
  'is_direct': False,
  'is_room': False,
  'is_type_room': False,
  'rent': 23,
  'deposit': 100,
  'is_deposit_only': False,
  'floor': '3층',
  '_floor': '3층',
  'floor_all': '4층',
  'local1': '세종시',
  'local2': '세종',
  'local3': '조치원읍',
  'title': '침산리 소재. 분리형의 풀옵션.조치원역까지 도보 10여분.',
 

In [40]:
#for check
item_list = [bang['item'] for bang in list(bigbang_collection.find(query)) if int(bang['item']['floor_all'][0]) >= 4 and int(bang['item']['floor'][0].replace("반", "0")) >= 3]
[(item['floor_all'], item['floor']) for item in item_list]

[('4층', '3층'),
 ('4층', '3층'),
 ('4층', '3층'),
 ('4층', '3층'),
 ('4층', '4층'),
 ('5층', '5층'),
 ('4층', '4층'),
 ('4층', '3층'),
 ('4층', '3층'),
 ('4층', '3층'),
 ('4층', '3층'),
 ('4층', '3층'),
 ('4층', '4층'),
 ('4층', '4층'),
 ('5층', '3층'),
 ('4층', '4층'),
 ('4층', '3층'),
 ('4층', '3층'),
 ('5층', '5층'),
 ('4층', '3층'),
 ('4층', '4층'),
 ('5층', '3층'),
 ('7층', '3층'),
 ('7층', '3층'),
 ('4층', '3층'),
 ('4층', '4층'),
 ('4층', '3층'),
 ('4층', '3층'),
 ('4층', '3층'),
 ('5층', '3층')]

## 2) Get the list of data WHERE size >= 33 or view count >= 50

In [44]:
query = {}

[bang['item'] for bang in list(bigbang_collection.find(query)) if int(bang['item']['size_m2']) >= 33 or int(bang['item']['view_count']) >= 50]

[{'id': 14187437,
  'images': [{'index': 0,
    'count': 1,
    'url': 'https://ic.zigbang.com/ic/items/14187437/1.jpg'},
   {'index': 1,
    'count': 2,
    'url': 'https://ic.zigbang.com/ic/items/14187437/2.jpg'},
   {'index': 2,
    'count': 3,
    'url': 'https://ic.zigbang.com/ic/items/14187437/3.jpg'},
   {'index': 3,
    'count': 4,
    'url': 'https://ic.zigbang.com/ic/items/14187437/4.jpg'},
   {'index': 4,
    'count': 5,
    'url': 'https://ic.zigbang.com/ic/items/14187437/5.jpg'},
   {'index': 5,
    'count': 6,
    'url': 'https://ic.zigbang.com/ic/items/14187437/6.jpg'},
   {'index': 6,
    'count': 7,
    'url': 'https://ic.zigbang.com/ic/items/14187437/7.jpg'}],
  'is_realestate': True,
  'is_direct': False,
  'is_room': False,
  'is_type_room': False,
  'rent': 23,
  'deposit': 100,
  'is_deposit_only': False,
  'floor': '3층',
  '_floor': '3층',
  'floor_all': '4층',
  'local1': '세종시',
  'local2': '세종',
  'local3': '조치원읍',
  'title': '침산리 소재. 분리형의 풀옵션.조치원역까지 도보 10여분.',
 

In [45]:
#for check
room_list = [bang['item'] for bang in list(bigbang_collection.find(query)) if int(bang['item']['size_m2']) >= 33 or int(bang['item']['view_count']) >= 50]
[(room['size_m2'], room['view_count']) for room in room_list]

[(19.83, '386'),
 (23.14, '223'),
 (36.36, '400'),
 (79.34, '100'),
 (19.83, '88'),
 (19.83, '110'),
 (16.53, '156'),
 (19.83, '209'),
 (19.83, '191'),
 (26.45, '77'),
 (22.48, '398'),
 (26.45, '79'),
 (19.83, '89'),
 (23.14, '68'),
 (23.14, '244'),
 (33.06, '110'),
 (19.83, '206'),
 (23.14, '239'),
 (26.45, '361'),
 (19.83, '477'),
 (26.45, '73'),
 (18.18, '50'),
 (33.06, '110'),
 (39.67, '518'),
 (26.45, '235'),
 (19.83, '192'),
 (19.83, '453'),
 (26.45, '89'),
 (29.75, '445'),
 (23.14, '107'),
 (49.59, '35'),
 (23.14, '57'),
 (26.45, '66'),
 (26.45, '123'),
 (26.45, '236'),
 (19.83, '150'),
 (19.83, '189'),
 (23.14, '182'),
 (29.75, '95'),
 (26.45, '51'),
 (23.14, '131'),
 (19.83, '299'),
 (39.67, '7'),
 (19.83, '402'),
 (26.45, '263'),
 (26.45, '128'),
 (19.83, '55'),
 (26.45, '181'),
 (26.45, '115'),
 (26.45, '81'),
 (26.45, '52')]

In [46]:
#or
query = {
    "$or" : [
        {
            "item.size_m2" : {
                "$gte" : 33,
            }
        },
        {
            "item.view_count" : {
                "gte" : 50,
            }
        }
    ]
}

list(bigbang_collection.find(query))

[{'_id': ObjectId('5c399b2ea6bb5b6cc4db1d37'),
  'title': '세종시 조치원읍 침산리',
  'header': False,
  'header_height': 0,
  'item': {'id': 14284983,
   'images': [{'index': 0,
     'count': 1,
     'url': 'https://ic.zigbang.com/ic/items/14284983/1.jpg'},
    {'index': 1,
     'count': 2,
     'url': 'https://ic.zigbang.com/ic/items/14284983/2.jpg'},
    {'index': 2,
     'count': 3,
     'url': 'https://ic.zigbang.com/ic/items/14284983/3.jpg'},
    {'index': 3,
     'count': 4,
     'url': 'https://ic.zigbang.com/ic/items/14284983/4.jpg'},
    {'index': 4,
     'count': 5,
     'url': 'https://ic.zigbang.com/ic/items/14284983/5.jpg'},
    {'index': 5,
     'count': 6,
     'url': 'https://ic.zigbang.com/ic/items/14284983/6.jpg'},
    {'index': 6,
     'count': 7,
     'url': 'https://ic.zigbang.com/ic/items/14284983/7.jpg'},
    {'index': 7,
     'count': 8,
     'url': 'https://ic.zigbang.com/ic/items/14284983/8.jpg'},
    {'index': 8,
     'count': 9,
     'url': 'https://ic.zigbang.com/ic